# 1. ETL

In [2]:
# Specify path to downloaded log file
import sys
import os

log_file_path = 'dbfs:/' + os.path.join('databricks-datasets', 'cs100', 'lab2', 'data-001', 'apache.access.log.PROJECT')
base_df = sqlContext.read.text(log_file_path)
base_df.printSchema()
base_df.show(truncate=False)

| field         | meaning                                                                |
| ------------- | ---------------------------------------------------------------------- |
| _remotehost_  | Remote hostname (or IP number if DNS hostname is not available).       |
| _rfc931_      | The remote logname of the user. We don't really care about this field. |
| _authuser_    | The username of the remote user, as authenticated by the HTTP server.  |
| _[date]_      | The date and time of the request.                                      |
| _"request"_   | The request, exactly as it came from the browser or client.            |
| _status_      | The HTTP status code the server sent back to the client.               |
| _bytes_       | The number of bytes (`Content-Length`) transferred to the client.      |

In [4]:
from pyspark.sql.functions import split, regexp_extract
split_df = base_df.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(truncate=False)

In [5]:
# are there any null values

bad_rows_df = split_df.filter(split_df['host'].isNull() |
                              split_df['timestamp'].isNull() |
                              split_df['path'].isNull() |
                              split_df['status'].isNull() |
                             split_df['content_size'].isNull())

print "number of rows with some null values", bad_rows_df.count()
print "this is a bad sign!!"



In [6]:
# let's find the null values:
nulls = {}
for c in split_df.columns:
  nulls[c] = split_df.filter(split_df[c].isNull()).count()  
nulls
print "It looks like it is coming from content_size column"

In [7]:
bad_content_size_df = base_df.filter(~base_df['value'].rlike('\d+$'))
bad_content_size_df.show(truncate=False)
bad_content_size_df.count()
# the count matches with the null values '-' indicates zero size 

In [8]:
cleaned_df = split_df.na.fill(0)
# let's find the null values after replacing them with 0:
nulls = {}
for c in split_df.columns:
  nulls[c] = split_df.filter(split_df[c].isNull()).count()  
print nulls
print "Horayyy. No null values"

In [9]:
month_map = {
  'Jan': 1, 'Feb': 2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7,
  'Aug':8,  'Sep': 9, 'Oct':10, 'Nov': 11, 'Dec': 12
}

def parse_clf_time(s):
    # We're ignoring time zone here.
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      month_map[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )
u_parse_time = udf(parse_clf_time)
logs_df = cleaned_df.select('*', u_parse_time(cleaned_df['timestamp']).cast('timestamp').alias('time')).drop('timestamp')
total_log_entries = logs_df.count()

print "total log entires" , total_log_entries
logs_df.show(10,False)
logs_df.cache()

# 2. Analysis

In [11]:
from pyspark.sql import functions as sql
content_size_summary_df = logs_df.describe(['content_size'])
logs_df.agg(sql.min(logs_df.content_size),sql.avg(logs_df.content_size),sql.max(logs_df.content_size)).show()



In [12]:
from pyspark.sql import functions
statuses = logs_df.groupBy('status').count().orderBy('count', ascending = False)

log_status = statuses.withColumn('log(count)', functions.log(statuses['count']))
display(log_status)

In [13]:
logs_df.groupBy('host').count().orderBy('count', ascending= False).show(5,truncate = False)
logs_df.groupBy('path').count().orderBy('count', ascending= False).show(5,truncate = False)

In [14]:
from pyspark.sql.functions import dayofmonth
host_day= logs_df.select(dayofmonth('time').alias('day'),'host').distinct()
daily_hosts_df = host_day.groupBy('day').count().orderBy('count',ascending=False)
daily_hosts_df.show(30)

from spark_notebook_helpers import prepareSubplot, np, plt, cm
days = []
unique_hosts = []
for r in daily_hosts_df.orderBy('day').collect():
  days.append(r[0])
  unique_hosts.append(r[1])
  
fig, ax = prepareSubplot(np.arange(0, 30, 5), np.arange(0, 5000, 1000))
colorMap = 'Dark2'
cmap = cm.get_cmap(colorMap)
plt.plot(days, unique_hosts, color=cmap(0), linewidth=3)
plt.axis([0, max(days), 0, max(unique_hosts)+500])
plt.xlabel('Day')
plt.ylabel('Hosts')
plt.axhline(linewidth=3, color='#999999')
plt.axvline(linewidth=2, color='#999999')
display(fig)


In [15]:
host_day_count  = logs_df.select(dayofmonth('time').alias('day'),'host').groupBy('host','day').count()
avg_daily_req_per_host_df = host_day_count.groupBy('day').avg('count').orderBy('day')
avg_daily_req_per_host_df.show()
display(avg_daily_req_per_host_df)
print 'No significant findings here'

In [16]:
print 'top 404 paths'
split_df.filter(split_df.status == '404').groupBy('path').count().orderBy('count', ascending = False).show(5,truncate = False)
print 'top 404 hosts'
split_df.filter(split_df.status == '404').groupBy('host').count().orderBy('count', ascending = False).show(5,truncate = False)

In [17]:
from pyspark.sql.functions import hour

perhour404 = logs_df.withColumn('hour',hour('time').cast('integer')).filter(split_df.status == '404').groupBy('hour').count().orderBy('hour')
display(perhour404)

# perday404 = logs_df.withColumn('day',dayofmonth('time').cast('integer')).filter(split_df.status == '404').groupBy('day').count().orderBy('day')
#perday404.collect()
# days = perday404.rdd.map(lambda r:r['day']).collect()
# errors = perday404.rdd.map(lambda r:r['count']).collect()
# fig, ax = prepareSubplot(np.arange(0, 20, 5), np.arange(0, 600, 100))
# colorMap = 'rainbow'
# cmap = cm.get_cmap(colorMap)
# plt.plot(days, errors, color=cmap(0), linewidth=3)
# plt.axis([0, max(days), 0, max(errors)])
# plt.xlabel('Day')
# plt.ylabel('404 Errors')
# plt.axhline(linewidth=3, color='#999999')
# plt.axvline(linewidth=2, color='#999999')
# display(fig)